# Systematic Literature Reviews


## Why are they useful?
1. Robust
2. Transparent
3. Repeatable

## What are the steps?
1. Identify a research question (e.g., How do energy models account for structural uncertainty?)
2. ???
3. Profit.

# ??? - How does one systematically search for papers?

The subject of this notebook is finding papers to include in your review. We will review two methods. 

1. Search engines.
2. Database API (Scopus)

## Method 1: Search Engines

You know this one.

## Method 2: Query a database

A more sophisticated way to search for papers.

[Scopus Database](https://www.scopus.com/search)


## Method 3: Database API (Scopus)

Scopus is the largest database of articles and a great place to start.

### Pre-requisites
1. A Scopus account (sign up through the institution).
2. An [API key](https://dev.elsevier.com/) (follow the instructions).
3. For full access (while at home), you will also need an institutional access token (these are hard to acquire).

In [1]:
import yaml

with open('es-api-key.yml', 'r') as f:
    data = yaml.load(f, Loader=yaml.SafeLoader)
     
api_key = data['apiKey']
inst_token = data['instToken']

## Python Requests

Fundamentally, creating an API call is the same as constructing a web address with a specific structure called a **well-known text** (WKT). 

`requests` is a Python library specifically for this use case.

We need: 

1. A base URL (the parent)
2. A set of "query" parameters (this changes across APIs -- read the docs for your specific API (e.g., EIA, NOAA, Twitter, etc.)).

[This page](https://dev.elsevier.com/documentation/ScopusSearchAPI.wadl#simple) lists the necessary query params.

In [2]:
import requests
import json # for reading the results.
import pandas as pd

In [3]:
SEARCH = "http://api.elsevier.com/content/search/scopus"

In [4]:
query = "TITLE-ABS-KEY ( \"method of characteristics\" ) AND TITLE-ABS-KEY ( \"neutron transport\" )"

In [5]:
index = 0
view = 'COMPLETE'
par = {'apikey': api_key, 'insttoken':inst_token, 
       'query': query, 'start': index,
       'httpAccept': 'application/json', 'view': view}

In [6]:
response = requests.get(SEARCH, params=par)

In [7]:
response

<Response [200]>

In [9]:
results = response.json()
print(results['search-results'].keys())

dict_keys(['opensearch:totalResults', 'opensearch:startIndex', 'opensearch:itemsPerPage', 'opensearch:Query', 'link', 'entry'])


In [10]:
results = results['search-results']
results['opensearch:totalResults']

'239'

In [13]:
df = pd.DataFrame(results['entry'])
df.head(2)

,@_fa,link,prism:url,dc:identifier,eid,dc:title,dc:creator,prism:publicationName,prism:issn,prism:eIssn,...,source-id,fund-acr,fund-no,fund-sponsor,openaccess,openaccessFlag,prism:issueIdentifier,prism:isbn,freetoread,freetoreadLabel
0,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85185561082,2-s2.0-85185561082,A memory-efficient neutron noise algorithm for...,Cosgrove P.,Annals of Nuclear Energy,03064549,18732100,...,29363,EPSRC,EP/T022159/1,Engineering and Physical Sciences Research Cou...,1,True,NaN,NaN,NaN,NaN
1,true,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85184710274,2-s2.0-85184710274,Time-dependent neutron transport method based ...,Song Q.,Annals of Nuclear Energy,03064549,18732100,...,29363,NSFC,12105170,National Natural Science Foundation of China,0,False,NaN,NaN,NaN,NaN


In [15]:
df['dc:description'].values[0]

'The neutron noise equation contains a fixed source term where the neutron angular flux is present. Deterministic noise solvers have tended to solve for and store this angular flux term in a preprocessing step. This can be a substantial memory burden as the angular flux is a function of space, energy, and angle. This can limit these solvers to smaller problems and can limit the obtainable angular resolution. This paper proposes solving the neutron noise equation and the standard eigenvalue neutron transport equation simultaneously, reducing the memory footprint of a noise solve while making frequency-domain method of characteristics solvers practical. This approach is demonstrated using the random ray method but is applicable to any sweep-based methods. Using the random ray method also has the advantage of exact geometry representation, no ray effects, and cheaper transport sweeps, all of which have been problematic. These advantages are illustrated on several noise problems.'

## A better way... 


`pybliometrics` abstracts the API calls 


```{note}
This library requires you to be on campus, even with the "STANDARD" view, unless you have an institutional access token.
```

The first time you run `pybliometrics` from the commandline, it will ask you to type in your access credentials (I have already done this).

In [16]:
from pybliometrics.scopus import ScopusSearch

x = ScopusSearch(query=query, view='COMPLETE')

In [18]:
x.get_results_size()

239

In [24]:
pyb_df = pd.DataFrame([doc._asdict() for doc in x.results])
display(len(pyb_df))
pyb_df.head(2)

239

,eid,doi,pii,pubmed_id,title,subtype,subtypeDescription,creator,afid,affilname,...,pageRange,description,authkeywords,citedby_count,openaccess,freetoread,freetoreadLabel,fund_acr,fund_no,fund_sponsor
0,2-s2.0-85185561082,10.1016/j.anucene.2024.110450,S0306454924001130,None,A memory-efficient neutron noise algorithm for...,ar,Article,Cosgrove P.,60120016,Department of Engineering,...,None,The neutron noise equation contains a fixed so...,Method of characteristics | Neutron noise | Ne...,0,1,None,None,EPSRC,EP/T022159/1,Engineering and Physical Sciences Research Cou...
1,2-s2.0-85184710274,10.1016/j.anucene.2023.110334,S0306454923006539,None,Time-dependent neutron transport method based ...,ar,Article,Song Q.,60025084,Shanghai Jiao Tong University,...,None,The 3D Method of Characteristics (MOC) has hig...,C5G7-TD | Method of characteristics | Neutron ...,0,0,None,None,NSFC,12105170,National Natural Science Foundation of China
